# Hospital Readmission Prediction - Prediction and analysis

**Objective:** Predict 30-day readmission risk using optimized XGBoost with clinical risk stratification

## 1. Setup

In [28]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# Configuration
import warnings
warnings.filterwarnings('ignore')

In [19]:
with open('ml_models/xgboost_final.pkl', 'rb') as f:
    model_package = pickle.load(f)

model = model_package['model']
optimal_threshold = model_package['optimal_threshold']
feature_names = model_package['feature_names']

print("Model loaded successfully")

Model loaded successfully


In [20]:
df = pd.read_csv('data/diabetic_data_preprocessed.csv')
X = df.drop('readmitted', axis=1)
y = df['readmitted']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

## 2. Prediction

In [21]:
y_proba = model.predict_proba(X_test)[:, 1]

y_pred = (y_proba >= optimal_threshold).astype(int)

# Performance report
print("MODEL PERFORMANCE")
print(classification_report(y_test, y_pred, 
                          target_names=['No Readmission', 'Readmission']))

MODEL PERFORMANCE
                precision    recall  f1-score   support

No Readmission       0.94      0.43      0.59     18083
   Readmission       0.15      0.79      0.25      2271

      accuracy                           0.47     20354
     macro avg       0.54      0.61      0.42     20354
  weighted avg       0.85      0.47      0.55     20354



## 3. Risk Stratification (3-Level System)

Classify patients into Low, Medium, and High risk categories for clinical decision-making.

In [22]:
def stratify_risk(probabilities, low_threshold=0.27, high_threshold=0.51):
    """
    Stratify patients into three risk categories based on readmission probability.
    
    Clinical interpretation:
    - Low Risk: < 15% probability
      → Standard discharge protocol
    - Medium Risk: 15-35% probability  
      → Enhanced monitoring, phone follow-up
    - High Risk: > 35% probability
      → Intensive case management, early clinic visit
    
    Args:
        probabilities: Array of predicted probabilities
        low_threshold: Upper bound for low risk (default: 0.15)
        high_threshold: Lower bound for high risk (default: 0.35)
    
    Returns:
        Array of risk categories: 0=Low, 1=Medium, 2=High
    """
    risk_categories = np.zeros(len(probabilities), dtype=int)
    risk_categories[probabilities >= low_threshold] = 1
    risk_categories[probabilities >= high_threshold] = 2
    return risk_categories

In [23]:
# Apply stratification
risk_strata = stratify_risk(y_proba)

# Distribution of risk categories
risk_distribution = pd.Series(risk_strata).value_counts().sort_index()
risk_labels = ['Low Risk', 'Medium Risk', 'High Risk']

print("RISK STRATIFICATION DISTRIBUTION")
for i, (count, pct) in enumerate(zip(risk_distribution.values, 
                                      risk_distribution.values / len(y_proba) * 100)):
    print(f"{risk_labels[i]:12} : {count:5} patients ({pct:5.2f}%)")

RISK STRATIFICATION DISTRIBUTION
Low Risk     :  3381 patients (16.61%)
Medium Risk  : 14812 patients (72.77%)
High Risk    :  2161 patients (10.62%)


In [33]:
risk_counts = list(risk_distribution.values)
risk_percentages = [count / len(y_proba) * 100 for count in risk_counts]

fig = go.Figure(data=[
    go.Bar(
        x=risk_labels,
        y=risk_counts,
        marker=dict(color=['#2ecc71', '#f39c12', '#e74c3c']),
        text=[f'{count}<br>({pct:.1f}%)' for count, pct in zip(risk_counts, risk_percentages)],
        textposition='outside',
        textfont=dict(size=11, family='Arial, sans-serif'),
    )
])

fig.update_layout(
    title=dict(
        text='Patient Distribution by Risk Category',
        font=dict(size=14, family='Arial, sans-serif')
    ),
    xaxis_title=dict(text='Risk Category', font=dict(size=12)),
    yaxis_title=dict(text='Number of Patients', font=dict(size=12)),
    width=1000,
    height=600,
    template='plotly_white'
)

fig.show()

## 4. Clinical Performance by Risk Category


In [25]:
print("ACTUAL READMISSION RATES BY RISK CATEGORY")


for i, label in enumerate(risk_labels):
    mask = risk_strata == i
    actual_rate = y_test[mask].mean() * 100
    n_patients = mask.sum()
    print(f"{label:12} : {actual_rate:5.2f}% actual readmission rate "
          f"(n={n_patients})")

ACTUAL READMISSION RATES BY RISK CATEGORY
Low Risk     :  4.53% actual readmission rate (n=3381)
Medium Risk  : 10.69% actual readmission rate (n=14812)
High Risk    : 24.71% actual readmission rate (n=2161)


## 6. ROI comparing to baseline model, xgboost first model or no model


In [31]:
# Cost parameters (in USD) - 3-level stratification
COST_NO_INTERVENTION = 0  # Low risk: no intervention
COST_LIGHT_MONITORING = 500  # Medium risk: phone follow-up
COST_INTENSIVE_MONITORING = 1500  # High risk: intensive case management

# Readmission costs based on intervention level
COST_SURPRISE_READMISSION = 22000 # No intervention
COST_PARTIAL_CONTROLLED_READMISSION = 16500  # Light monitoring
COST_CONTROLLED_READMISSION = 13500  # Intensive monitoring

# Select random sample of 1000 patients
np.random.seed(42)
sample_indices = np.random.choice(len(X_test), size=1000, replace=False)
X_sample = X_test.iloc[sample_indices]
y_sample = y_test.iloc[sample_indices]

print(f"Sample size: {len(y_sample)}")
print(f"Actual readmissions in sample: {y_sample.sum()} ({y_sample.mean()*100:.1f}%)\n")

# Function to calculate total cost with 3-level stratification
def calculate_total_cost_stratified(y_true, probabilities):
    """
    Calculate total cost based on 3-level risk stratification.
    
    Strategy based on predicted risk:
    - Low Risk (< 5%): No intervention ($0)
      * If readmitted: Surprise readmission ($22,000)
    - Medium Risk (10-25%): Light monitoring ($500)
      * If readmitted: Partially controlled ($16,500)
    - High Risk (> 25%): Intensive monitoring ($1,500)
      * If readmitted: Controlled readmission ($13,500)
    """
    risk_categories = stratify_risk(probabilities)
    total_cost = 0
    
    for true_val, risk_cat in zip(y_true, risk_categories):
        if risk_cat == 0:  # Low risk
            total_cost += COST_NO_INTERVENTION
            if true_val == 1:
                total_cost += COST_SURPRISE_READMISSION
        elif risk_cat == 1:  # Medium risk
            total_cost += COST_LIGHT_MONITORING
            if true_val == 1:
                total_cost += COST_PARTIAL_CONTROLLED_READMISSION
        else:  # High risk
            total_cost += COST_INTENSIVE_MONITORING
            if true_val == 1:
                total_cost += COST_CONTROLLED_READMISSION
    
    return total_cost

# Scenario 1: No model (treat everyone as low-risk)
proba_no_model = np.zeros(len(y_sample))  # All probabilities = 0
cost_no_model = calculate_total_cost_stratified(y_sample, proba_no_model)

# Scenario 2: Baseline logistic regression
with open('ml_models/logistic_regression_baseline.pkl', 'rb') as f:
    model_baseline = pickle.load(f)
proba_baseline = model_baseline.predict_proba(X_sample)[:, 1]
cost_baseline = calculate_total_cost_stratified(y_sample, proba_baseline)

# Scenario 3: XGBoost initial
with open('ml_models/xgboost_optimized.pkl', 'rb') as f:
    model_xgb_initial = pickle.load(f)
proba_xgb_initial = model_xgb_initial.predict_proba(X_sample)[:, 1]
cost_xgb_initial = calculate_total_cost_stratified(y_sample, proba_xgb_initial)

# Scenario 4: XGBoost final (already loaded as 'model')
proba_final = model.predict_proba(X_sample)[:, 1]
cost_final = calculate_total_cost_stratified(y_sample, proba_final)

# Calculate savings vs no model
savings_baseline = ((cost_no_model - cost_baseline) / cost_no_model) * 100
savings_xgb_initial = ((cost_no_model - cost_xgb_initial) / cost_no_model) * 100
savings_final = ((cost_no_model - cost_final) / cost_no_model) * 100

# Display results
results_df = pd.DataFrame({
    'Model': ['No Model', 'Baseline (Logistic)', 'XGBoost Initial', 'XGBoost Final'],
    'Total Cost ($)': [cost_no_model, cost_baseline, cost_xgb_initial, cost_final],
    'Savings vs No Model (%)': [0, savings_baseline, savings_xgb_initial, savings_final]
})

print("\n" + "="*80)
print("ROI ANALYSIS - 3-Level Risk Stratification (1,000 patients)")
print("="*80)
print("\nCost Structure:")
print(f"  • Low Risk: ${COST_NO_INTERVENTION:,} intervention | ${COST_SURPRISE_READMISSION:,} if readmitted")
print(f"  • Medium Risk: ${COST_LIGHT_MONITORING:,} intervention | ${COST_PARTIAL_CONTROLLED_READMISSION:,} if readmitted")
print(f"  • High Risk: ${COST_INTENSIVE_MONITORING:,} intervention | ${COST_CONTROLLED_READMISSION:,} if readmitted")
print("\n" + results_df.to_string(index=False))
print("="*80)

colors = ['#d62728', '#ff7f0e', '#2ca02c', '#1f77b4']

# Bar chart - Total costs
fig1 = go.Figure(data=[
    go.Bar(
        x=results_df['Model'],
        y=results_df['Total Cost ($)'] / 1000,
        marker=dict(color=colors),
        text=[f'${val:.0f}k' for val in results_df['Total Cost ($)'] / 1000],
        textposition='outside',
        textfont=dict(size=10),
    )
])

fig1.update_layout(
    title=dict(
        text='Total Cost by Model<br>3-Level Risk Stratification (1,000 patients)',
        font=dict(size=12)
    ),
    yaxis_title='Total Cost (thousands $)',
    height=500,
    width=1000,
    template='plotly_white',
    showlegend=False
)

fig1.update_xaxes(tickangle=15)
fig1.show()

# Bar chart - Savings percentage
fig2 = go.Figure(data=[
    go.Bar(
        x=results_df['Model'][1:],
        y=results_df['Savings vs No Model (%)'][1:],
        marker=dict(color=colors[1:]),
        text=[f'{val:.1f}%' for val in results_df['Savings vs No Model (%)'][1:]],
        textposition='outside',
        textfont=dict(size=10),
    )
])

fig2.update_layout(
    title=dict(
        text='Cost Savings vs No Model<br>(% reduction)',
        font=dict(size=12)
    ),
    yaxis_title='Savings (%)',
    height=500,
    width=1000,
    template='plotly_white',
    showlegend=False
)

fig2.update_xaxes(tickangle=15)
fig2.add_hline(y=0, line=dict(color='black', width=0.8))
fig2.show()

# Summary
print(f"\n📊 KEY FINDINGS:")
print(f"   • Best model: {results_df.loc[results_df['Total Cost ($)'].idxmin(), 'Model']}")
print(f"   • Maximum savings: {results_df['Savings vs No Model (%)'].max():.1f}%")
print(f"   • Cost reduction: ${(cost_no_model - cost_final):,.0f}")


Sample size: 1000
Actual readmissions in sample: 125 (12.5%)


ROI ANALYSIS - 3-Level Risk Stratification (1,000 patients)

Cost Structure:
  • Low Risk: $0 intervention | $22,000 if readmitted
  • Medium Risk: $500 intervention | $16,500 if readmitted
  • High Risk: $1,500 intervention | $13,500 if readmitted

              Model  Total Cost ($)  Savings vs No Model (%)
           No Model         2750000                 0.000000
Baseline (Logistic)         2690000                 2.181818
    XGBoost Initial         2691500                 2.127273
      XGBoost Final         2546500                 7.400000



📊 KEY FINDINGS:
   • Best model: XGBoost Final
   • Maximum savings: 7.4%
   • Cost reduction: $203,500

📋 INTERVENTION DISTRIBUTION (XGBoost Final):
   • Low Risk    :  174 patients ( 17.4%)
   • Medium Risk :  697 patients ( 69.7%)
   • High Risk   :  129 patients ( 12.9%)


## 7. Feature Importance for Clinical Interpretation

In [32]:
feature_importance = model.get_feature_importance(feature_names)
importance_df = pd.DataFrame({
    'feature': feature_importance.keys(),
    'importance': feature_importance.values()
}).sort_values('importance', ascending=False).head(15)

fig = go.Figure(data=[
    go.Bar(
        x=importance_df['importance'],
        y=importance_df['feature'],
        orientation='h',
        marker=dict(color='#1f77b4')
    )
])

fig.update_layout(
    title=dict(
        text='Top 15 Most Important Features for Readmission Prediction',
        font=dict(size=14)
    ),
    xaxis_title='Importance',
    yaxis_title='',
    height=600,
    width=1000,
    template='plotly_white',
    showlegend=False
)

fig.update_yaxes(autorange='reversed')
fig.show()